In [2]:
from common import *

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


In [3]:
with open(top_dir + 'data/chance/days_order.txt', 'r') as file:
    days = pd.Series([int(x) for x in file.read().splitlines()], name='Day')
n_days = len(days)
days_z = dg.Day.unique()

In [4]:
n_days

258

In [114]:
if __name__ == '__main__':
    days_a, days_b, days_c = days[:n_days // 3], days[n_days // 3:(2 * n_days) // 3], days[(2 * n_days) // 3:]
    print(len(days_a), len(days_b), len(days_c))

86 86 86


In [44]:
def split2(dh, ho=True, repeat=1):
    if not isinstance(dh.columns, pd.MultiIndex) and 'Day' in dh:
        index = dh.Day
    elif isinstance(dh, pd.Series) and dh.name == 'Day':
        index = dh
    else:
        assert dh.index.name == 'Day'
        index = dh.index
    if ho:
        tc, ho = days[:176], days[176:]
    else:
        tc = days
    t0c1 = [(tc.index // (1 << i) % 2).values.astype(bool) for i in range(repeat)]
    #
    indexify = lambda a: index.isin(set(a)) if isinstance(index, pd.Series) else pd.Series(index.isin(set(a)), index=index)
    s = O(type=2, repeat=repeat)
    s.tc = indexify(tc)
    s.trcv = [(indexify(tc[~x]), indexify(tc[x])) for x in t0c1]
    if repeat == 1:
        s.tr = indexify(tc[~t0c1[0]])
        s.cv = indexify(tc[t0c1[0]])
    if ho is not None and ho is not False and ho is not 0:
        s.ho = indexify(ho)
    s.fn = indexify(days_z)
    return s

In [45]:
def split3(dh, repeat=1):
    if not isinstance(dh.columns, pd.MultiIndex) and 'Day' in dh:
        index = dh.Day
    elif isinstance(dh, pd.Series) and dh.name == 'Day':
        index = dh
    else:
        assert dh.index.name == 'Day'
        index = dh.index
    tc, ho = days[:180], days[180:]
    tc3 = [(tc.index // 3 ** i % 3).values for i in range(repeat)]
    #
    indexify = lambda a: index.isin(set(a)) if isinstance(index, pd.Series) else pd.Series(index.isin(set(a)), index=index)
    s = O(type=3, repeat=repeat)
    s.tc = indexify(tc)
    s.trcv = [] #(indexify(tc[x < 2]), indexify(tc[x == 2])) for x in tc3]
    for r in range(repeat):
        s.trcv += [(indexify(tc[tc3[r] != i]), indexify(tc[tc3[r] == i])) for i in range(3)]
    s.ho = indexify(ho)
    s.fn = indexify(days_z)
    return s